# OHLCV - Cryptocurrency Data Collection

## Objective

Get OHLCV information for the following CryptoCurrencies
*['Binance Coin', 'Bitcoin', 'EOS', 'Ethereum', 'Litecoin','Stellar', 'TRON', 'XRP', 'Bitcoin Cash']*

1. We have used cryptocompare api to get CryptoCurrencies OHLCV information 
2. The data is collected from 1st January 2018 till 27th Feb 2019 [1 Year] for all 9 CryptoCurrencies

In [14]:
#OHLCV Information
import urllib.request
import json
import pandas as pd


complete_df=pd.DataFrame()
df = pd.read_json('Hourly-Processed-Data/crypto.json').set_index('symbol')

#add &toTs=x to get all data up to the timestamp x
#this command will return last 2000 samples
#contents = urllib.request.urlopen("https://min-api.cryptocompare.com/data/histohour?fsym=BTC&tsym=USDT&e=binance&limit=2000").read()

finalSample = 1552608000	#this is the final timestamp for now. This corresponds to roughly midnight on 1/22/19
numSampleSets = 5	#number of sets of "entriesPerSample" samples
entriesPerSample = 2000 #number of samples per call (2000 is max)
coins =['BNB', 'BTC', 'EOS', 'ETH', 'LTC', 'XLM', 'USDT', 'TRX', 'XRP', 'BCH']

samplingIntervalTS = entriesPerSample*3600	#this is the number of seconds in each sampleSet

for coin in coins:
    crypto_name = df.loc[coin]['name']
    for sampleCall in range(1,numSampleSets+1):
        timeToCall = finalSample - samplingIntervalTS*(numSampleSets - sampleCall)
        print('\x1b[1;31m',coin,'ohlcv data of size ',sampleCall*entriesPerSample , 'have been downloaded\x1b[0m')

        contents = urllib.request.urlopen("https://min-api.cryptocompare.com/data/histohour?fsym="+coin+"&tsym=USDT&e=binance&limit=2000&toTs=" + str(timeToCall)).read()
        jsonResponse = json.loads(contents.decode('utf-8'))
        rawData = pd.DataFrame.from_records(jsonResponse['Data'])
        rawData['symbol'] = coin
        rawData['crypto_name'] = crypto_name
        complete_df=complete_df.append(rawData,ignore_index=True)
complete_df['new_time'] = pd.to_datetime(complete_df['time'],unit='s')
save_df = complete_df[[ 'crypto_name','symbol', 'new_time','open','high',\
                       'low','close','volumefrom', 'volumeto']]\
                    .rename({'new_time':'created_time','symbol':'asset_name'},axis=1)
#save_df.to_json('crypto_hist.json',orient='records',date_format='iso')
save_df.head(4)


 BNB ohlcv data of size  2000 have been downloaded
 BNB ohlcv data of size  4000 have been downloaded
 BNB ohlcv data of size  6000 have been downloaded
 BNB ohlcv data of size  8000 have been downloaded
 BNB ohlcv data of size  10000 have been downloaded


,crypto_name,asset_name,created_time,open,high,low,close,volumefrom,volumeto
0,Binance Coin,BNB,2018-01-22 08:00:00,14.14,14.44,14.00,14.10,105978.95,1507209.31
1,Binance Coin,BNB,2018-01-22 09:00:00,14.10,14.35,13.85,14.20,83496.47,1184968.02
2,Binance Coin,BNB,2018-01-22 10:00:00,14.20,14.24,14.03,14.05,33415.74,471694.25
3,Binance Coin,BNB,2018-01-22 11:00:00,14.05,14.11,13.41,13.43,110293.99,1515195.81


In [19]:
print('\x1b[1;31m OHLCV Data for CryptoCurrency \x1b[0m')
crypto_data = pd.read_json('Hourly-Processed-Data/crypto_hist.json')
crypto_data.groupby('crypto_name').asset_name.count().reset_index()


 OHLCV Data for CryptoCurrency 


,crypto_name,asset_name
0,Binance Coin,10005
1,Bitcoin,10005
2,Bitcoin Cash,10005
3,EOS,10005
4,Ethereum,10005
5,Litecoin,10005
6,Stellar,10005
7,TRON,10005
8,XRP,10005
